<a href="https://colab.research.google.com/github/syunsuke1024/Portfolio-1-competitive-programming/blob/master/1006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

In [2]:
import numpy as np
import pandas as pd

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

In [3]:
DEBUG = False

train = pd.read_csv('/content/drive/MyDrive/kaggle-paper2021-09-23/train.csv')
test = pd.read_csv('/content/drive/MyDrive/kaggle-paper2021-09-23/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/kaggle-paper2021-09-23/sample_submission.csv')

if DEBUG:
    train = train[:80*100]

In [4]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()

    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)

    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df

train = add_features(train)
test = add_features(test)

In [5]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

In [6]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [7]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [8]:
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
class WarmupExponentialDecay(Callback):
    def __init__(self,lr_base=0.0002,lr_min=0.0,decay=0,warmup_epochs=0):
        self.num_passed_batchs = 0   #一个计数器
        self.warmup_epochs=warmup_epochs  
        self.lr=lr_base #learning_rate_base
        self.lr_min=lr_min #最小的起始学习率,此代码尚未实现
        self.decay=decay  #指数衰减率
        self.steps_per_epoch=0 #也是一个计数器
        
    def on_batch_begin(self, batch, logs=None):
        # params是模型自动传递给Callback的一些参数
        if self.steps_per_epoch==0:
            #防止跑验证集的时候呗更改了
            if self.params['steps'] == None:
                self.steps_per_epoch = np.ceil(1. * self.params['samples'] / self.params['batch_size'])
            else:
                self.steps_per_epoch = self.params['steps']
        if self.num_passed_batchs < self.steps_per_epoch * self.warmup_epochs:
            K.set_value(self.model.optimizer.lr,
                        self.lr*(self.num_passed_batchs + 1) / self.steps_per_epoch / self.warmup_epochs)
        else:
            K.set_value(self.model.optimizer.lr,
                        self.lr*((1-self.decay)**(self.num_passed_batchs-self.steps_per_epoch*self.warmup_epochs)))
        self.num_passed_batchs += 1

    def on_epoch_begin(self,epoch,logs=None):
        #用来输出学习率的,可以删除
        print("learning_rate:",K.get_value(self.model.optimizer.lr))

In [ ]:
EPOCH = 300
BATCH_SIZE = 1024
NUM_FOLDS = 10

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=3)
    test_preds = []
    train_preds = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        if fold <= 3:
          continue
        else:
            
          X_train, X_valid = train[train_idx], train[test_idx]
          y_train, y_valid = targets[train_idx], targets[test_idx]
          model = keras.models.Sequential([
              keras.layers.Input(shape=train.shape[-2:]),
              keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
              keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
  #             keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
              keras.layers.Dense(128, activation='selu'),
  #             keras.layers.Dropout(0.1),
              keras.layers.Dense(1),
          ])
          model.compile(optimizer="adam", loss="mae")

  #         scheduler = ExponentialDecay(1e-3, 40*((len(train)*0.8)/BATCH_SIZE), 1e-5)
  #         lr = LearningRateScheduler(scheduler, verbose=1)
          lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
  #         lr = WarmupExponentialDecay(lr_base=1e-3, decay=1e-5, warmup_epochs=30)
          es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
      
          checkpoint_filepath = f"/content/drive/MyDrive/kaggle-paper2021-09-23/1006/003folds{fold}.hdf5"
          sv = keras.callbacks.ModelCheckpoint(
              checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
              save_weights_only=False, mode='auto', save_freq='epoch',
              options=None
          )

          model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
          #model.save(f'Fold{fold+1} RNN Weights')
          test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())
          submission["pressure"] = sum(test_preds)/1
          submission.to_csv('/content/drive/MyDrive/kaggle-paper2021-09-23/1006/test_003-'+str(fold)+".csv", index=False)
          test_preds = []
          train_preds.append(model.predict(train).squeeze().reshape(-1, 1).squeeze())
          t = pd.read_csv('/content/drive/MyDrive/kaggle-paper2021-09-23/train.csv')
          t["pressure"] = sum(train_preds)/1
          t["pressure"].to_csv('/content/drive/MyDrive/kaggle-paper2021-09-23/1006/train_003-'+str(fold)+".csv", index=False)
          train_preds = []


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.22.252.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.22.252.90:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


--------------- > Fold 1 < ---------------
--------------- > Fold 2 < ---------------
--------------- > Fold 3 < ---------------
--------------- > Fold 4 < ---------------
--------------- > Fold 5 < ---------------
Epoch 1/300
67/67 [==============================] - 104s 1s/step - loss: 2.4565 - val_loss: 1.3938

Epoch 00001: val_loss improved from inf to 1.39383, saving model to /content/drive/MyDrive/kaggle-paper2021-09-23/1006/003folds4.hdf5
Epoch 2/300
67/67 [==============================] - 43s 640ms/step - loss: 0.9065 - val_loss: 0.7103

Epoch 00002: val_loss improved from 1.39383 to 0.71031, saving model to /content/drive/MyDrive/kaggle-paper2021-09-23/1006/003folds4.hdf5
Epoch 3/300
67/67 [==============================] - 43s 640ms/step - loss: 0.6872 - val_loss: 0.6313

Epoch 00003: val_loss improved from 0.71031 to 0.63133, saving model to /content/drive/MyDrive/kaggle-paper2021-09-23/1006/003folds4.hdf5
Epoch 4/300
67/67 [==============================] - 43s 640ms/step 